In [ ]:
import pandas as pd
import numpy as np
import os
import typing
import matplotlib.pyplot as plt
import time

from sklearn.preprocessing import MinMaxScaler
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.statespace.sarimax import SARIMAX
from sklearn.metrics import mean_squared_error
from math import sqrt

In [ ]:
#Smooth curve
def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
 
    
    
    y_smooth=np.convolve(y, box, 
                           mode='valid'
                          )
   
    return y_smooth

In [ ]:
def train_test_split(dataset, ratio):
    scaler = MinMaxScaler()

    
    train_size = int(len(dataset) * ratio)
    test_size = len(dataset) - train_size

    train  = dataset[0:train_size]
    test   = dataset[train_size:]
    return train,test

In [ ]:
def graph_predictions_sarima_append_dontfit(dataset,ratio,step=1,ordd=(5,1,0),Sorder=(5,1,0,45),smooth_flag=0):
    
    #try using SARIMA, change modelling options method conv and low memory 
    print("parameters:", ordd)
    print("seasonal parameters:", Sorder)
    
    plt.figure(figsize=(30,15))
    start = time.time()
    train,test = train_test_split(dataset,ratio)
    history = [x for x in train]
    predictions = list()
    # predictions=[x for x in train] #the prediction starts of lagged from history..
    # walk-forward validation
#     print("starting loop")
    model = ARIMA(history, order=ordd,seasonal_order=Sorder)
#         print("itareation #",t/step, "pre model.fit")
    print("start training")
    
#     model_fit = model.fit()#(method='innovations_mle', low_memory=True, cov_type='none')
    model_fit = model.fit(method='statespace', low_memory=True, cov_type='none')
#     model_fit = model.fit(cov_type='robust_approx')
    print("start predicting")
    output = model_fit.forecast(len(test))



    predictions.extend(output)


    plt.plot(test)
    # plt.plot(train)
    # plt.plot(dataset)
    # print(len(dataset))
    print("test: ", len(test))
    print("predictions: ", len(predictions))
    if smooth_flag:
        predictions_ma=[]
        predictions_ma.append(smooth(predictions,30))
        plt.plot(predictions_ma[0], color='red')
    
    else: 
        plt.plot(predictions, color='red')
    # plt.plot(, color='blu')
    plt.grid()
    plt.ylim([20,65])
    
    try:
        MSE = mean_squared_error(test,predictions)
        RMSE = sqrt(MSE)
        print("mean squared error= ",MSE)
    
        print("RMSE= " ,RMSE)
    except:
        print("error calculating, mismatch in dimentions")
        ...
    
    end = time.time()
    print("time taken: ",end - start)
#     plt.rcParams['figure.figsize'] = [1000/2.54, 800/2.54]
    plt.show()
    # print(len(predictions))

    
    # print(predictions_ma)

    # return predictions
    return predictions, test, RMSE,ordd,Sorder,model_fit;


In [ ]:
ds = pd.DataFrame()
PTR=[]
"""
"""
number_of_days = 0
for file_name in os.listdir():

    try:
#         print(file_name)
        temp = pd.read_csv( file_name, index_col='dateandtime',parse_dates=True)
        ds = ds.append(temp)
        number_of_hours+=1
    except:
        ...
number_of_days= number_of_hours/25

In [ ]:
temp = pd.pivot_table(ds,index = 'dateandtime', values ='spdK/m' , columns ='edge')

temp = temp.groupby([
            pd.Grouper(level='dateandtime'
                       , freq = '16T'  
                      )]
          ).mean()


In [ ]:
periodicity= int(len(smoothed_df[0])/int(number_of_days))

In [ ]:
limitPer = len(temp) *0.20
temp = temp.dropna(thresh=limitPer, axis=1)
temp.count()

In [ ]:
df = temp.fillna(temp.max())

In [ ]:
df = df.loc[:, (temp != 0).any(axis=0)]

In [ ]:
smoothed_df = []
for cntrDf in range(0, df.shape[-1]):
    
    smoothed_df.append((df.values[:,cntrDf]))



In [ ]:
plt.rcParams["figure.figsize"] = (30,15)
# plt.plot(smooth(df.values[:,0],40), 'r-', lw=2)
plt.plot(smoothed_df[0], 'b-', lw=2)
plt.ylim([0 , 90])
plt.grid()


In [ ]:
PTR.append(graph_predictions_sarima_append_dontfit(smoothed_df[0],0.7,3,(5, 0, 1) ,(3, 0, 1, periodicity), 0))#16T#remove optmization? 